stock news reaction period is generally 2 days, therefore base predictions off sentiment of articles from last 2 days
try stocknews for provided sentiment analysis of headlines
finvizfinance for stock news articles to analyze
try huggingface for pretrained financial news sentiment models
use kaggle data to train a new financial news sentiment model
use yfinance for market data

# Sentiment Analysis of Financial News

In [37]:
import numpy as np
import pandas as pd
import pickle as pk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
import torch
from torch.utils.data.dataset import Dataset


In [19]:
df = pd.read_csv("../data/data.csv")


#downsize data for testing, use full data for model training
df = df.head(500)

df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [38]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["Sentence"])


In [ ]:
tensor = torch.tensor()

ValueError: too many dimensions 'str'

In [ ]:
class financeTextDataset(Dataset):
    def __init__(self, dataframe):
        self.wordEmbeddings = CountVectorizer().fit_transform(dataframe["Sentence"])
        self.labels = dataframe["Sentiment"]
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.wordEmbeddings[index], self.labels[index]